In [16]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

import nltk.corpus

from nltk.corpus import RegexpTokenizer
from nltk.corpus import stopwords

nltk.download('stopwords')
# from keras.layers import Dense
# from keras.layers import Embedding
# from keras.layers import SimpleRNN
# from keras.layers import LSTM
# from keras.models import Sequential

# from keras.preprocessing import sequence
# from keras.preprocessing.sequence import pad_sequences
# from keras.preprocessing.text import Tokenizer

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from transformers import get_scheduler



import json
import string
import warnings

stopset = stopwords.words('english') + list(string.punctuation)
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yanivbronshtein/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df = df.groupby(by='category').head(1000)
df['summary'] = df['headline'] + '. ' +  df['short_description']
df = df[['summary','category']]

In [3]:
category_list = ['POLITICS', 'ENTERTAINMENT', 'TRAVEL', 'BUSINESS', 'SPORTS', 'RELIGION']
df_news = df[df['category'].isin(category_list)]

In [4]:
df_news.head()

,summary,category
1,Will Smith Joins Diplo And Nicky Jam For The 2...,ENTERTAINMENT
2,Hugh Grant Marries For The First Time At Age 5...,ENTERTAINMENT
3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,ENTERTAINMENT
4,Julianna Margulies Uses Donald Trump Poop Bags...,ENTERTAINMENT
5,Morgan Freeman 'Devastated' That Sexual Harass...,ENTERTAINMENT


# Let's try the pd.get_dummies() approach instead

In [5]:
df_news2 = pd.get_dummies(df_news, columns=['category'])

In [6]:
df_news2.head(1)

,summary,category_BUSINESS,category_ENTERTAINMENT,category_POLITICS,category_RELIGION,category_SPORTS,category_TRAVEL
1,Will Smith Joins Diplo And Nicky Jam For The 2...,0,1,0,0,0,0


In [8]:
all_data = df_news2['summary']
all_labels = df_news2.drop(columns=['summary'])
X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, train_size=.8, random_state=1)


# Check matching dimensions

In [9]:
assert(len(X_train) == len(y_train))
print('The number of samples in the training data and training labels match.')

The number of samples in the training data and training labels match.


In [10]:
assert(len(X_test) == len(y_test))
print('The number of samples in the test data and test labels match.')

The number of samples in the test data and test labels match.


# tfidf vectorize

In [11]:
vectorizer = TfidfVectorizer() #make Tfidf Vectorizer
tfidf_encodings = vectorizer.fit_transform(X_train) #encode the text
                   
# df['tfidf'] = list(tfidf_encodings.toarray()) #vectorized texts to dense list format for storage in dataframe

# vectors_for_training = np.array(df['tfidf'].tolist()) #get the vectors back out of the dataframe for use in something else

# Train RNN's

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])


In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [74]:
max_features = 10000

model = Sequential()
model.add(Embedding(max_features, 100))
model.add(SimpleRNN(32))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(tfidf_encodings, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_test, y_test))

# Error: Cast string to float is not supported

Epoch 1/10


UnimplementedError:  Cast string to float is not supported
	 [[node sequential_2/Cast
 (defined at /Users/yanivbronshtein/opt/anaconda3/lib/python3.7/site-packages/keras/engine/functional.py:671)
]] [Op:__inference_train_function_3358]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_2/Cast:
In[0] IteratorGetNext (defined at /Users/yanivbronshtein/opt/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:866)

Operation defined at: (most recent call last)
>>>   File "/Users/yanivbronshtein/opt/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
>>>     "__main__", mod_spec)
>>> 
>>>   File "/Users/yanivbronshtein/opt/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/Users/yanivbronshtein/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/Users/yanivbronshtein/opt

In [12]:
all_data = df_news['summary']
all_labels = df_news['category']

print(all_data[:2], '\n')
print(all_labels[:2], '\n')

print('Total number of reviews:', len(all_data))
print('Total number of labels:', len(all_labels))

1    Will Smith Joins Diplo And Nicky Jam For The 2...
2    Hugh Grant Marries For The First Time At Age 5...
Name: summary, dtype: object 

1    ENTERTAINMENT
2    ENTERTAINMENT
Name: category, dtype: object 

Total number of reviews: 6000
Total number of labels: 6000


In [ ]:
assert(len(X_train) == len(y_train))
print('The number of samples in the training data and training labels match.')

In [ ]:
assert(len(X_test) == len(y_test))
print('The number of samples in the test data and test labels match.')

In [ ]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train); labels = model.predict(X_test)

In [ ]:
mat = confusion_matrix(y_test, labels)
plt.figure(figsize=(2,2))

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))

plt.xlabel('True Labeling'); plt.ylabel('Predicted Labeling')

In [ ]:
print('Microaveraged F1 score:', f1_score(y_test, labels, average='micro'))
print('Macroaveraged F1 score:', f1_score(y_test, labels, average='macro'))

In [ ]:
model = make_pipeline(TfidfVectorizer(), MLPClassifier(verbose=True))
model.fit(X_train, y_train); labels = model.predict(X_test)

In [17]:
MLPClassifier?

In [ ]:
mat = confusion_matrix(y_test, labels)
plt.figure(figsize=(2,2))

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))

plt.xlabel('true label'); plt.ylabel('predicted label')

In [ ]:
print('Microaveraged F1 score:', f1_score(y_test, labels, average='micro'))
print('Macroaveraged F1 score:', f1_score(y_test, labels, average='macro'))

In [ ]:
model = make_pipeline(TfidfVectorizer(), MLPClassifier(hidden_layer_sizes=(100,100),verbose=True))
model.fit(X_train, y_train); labels = model.predict(X_test)

In [ ]:
mat = confusion_matrix(y_test, labels)
plt.figure(figsize=(2,2))

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))

plt.xlabel('true label'); plt.ylabel('predicted label')

In [ ]:
print('Microaveraged F1 score:', f1_score(y_test, labels, average='micro'))
print('Macroaveraged F1 score:', f1_score(y_test, labels, average='macro'))

In [ ]:
model = make_pipeline(TfidfVectorizer(), MLPClassifier(hidden_layer_sizes=(100,100,100),verbose=True))
model.fit(X_train, y_train); labels = model.predict(X_test)

In [ ]:
mat = confusion_matrix(y_test, labels)
plt.figure(figsize=(2,2))

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))

plt.xlabel('true label'); plt.ylabel('predicted label')

In [ ]:
print('Microaveraged F1 score:', f1_score(y_test, labels, average='micro'))
print('Macroaveraged F1 score:', f1_score(y_test, labels, average='macro'))

In [ ]:
mat = confusion_matrix(y_test, labels)
plt.figure(figsize=(2,2))

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))

plt.xlabel('true label'); plt.ylabel('predicted label')

In [ ]:
print('Microaveraged F1 score:', f1_score(y_test, labels, average='micro'))
print('Macroaveraged F1 score:', f1_score(y_test, labels, average='macro'))

In [ ]:
glove_file = datapath('C:/Users/benja/Downloads/glove.6B.100d.txt')
word2vec_text_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_text_file)

In [ ]:
wv = KeyedVectors.load_word2vec_format(word2vec_text_file)

In [ ]:
def calc_doc_embedding(doc, embeddings, embed_size, OOV_embed):
    word_count = 0
    doc_embed = np.zeros(embed_size, dtype=float)
    
    for word in doc.split():
        if word not in stopset:
            if (word in embeddings):
                word_embedding = embeddings[word]
                
            elif word in OOV_embed:
                word_embedding = OOV_embed[word]
                
            else:
                word_embedding = np.random.random(embed_size)
                OOV_embed[word] = word_embedding
                
            doc_embed += word_embedding
            word_count += 1
            
    doc_embed /= word_count
    return doc_embed

In [ ]:
OOV_embed = {}
X_train_embed = np.vstack([calc_doc_embedding(doc, wv, 100, OOV_embed) for doc in X_train])
X_test_embed = np.vstack([calc_doc_embedding(doc, wv, 100, OOV_embed) for doc in X_test])

In [ ]:
model = MLPClassifier(verbose=True)
model.fit(X_train_embed, y_train); labels = model.predict(X_test_embed)

In [ ]:
mat = confusion_matrix(y_test, labels)
plt.figure(figsize=(2,2))

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))

plt.xlabel('True Labeling'); plt.ylabel('Predicted Labeling')

In [ ]:
print('Microaveraged F1 score:', f1_score(y_test, labels, average='micro'))
print('Macroaveraged F1 score:', f1_score(y_test, labels, average='macro'))

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(100,100), verbose=True)
model.fit(X_train_embed, y_train); labels = model.predict(X_test_embed)

In [ ]:
mat = confusion_matrix(y_test, labels)
plt.figure(figsize=(2,2))

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))

plt.xlabel('True Labeling'); plt.ylabel('Predicted Labeling')

In [ ]:
print('Microaveraged F1 score:', f1_score(y_test, labels, average='micro'))
print('Macroaveraged F1 score:', f1_score(y_test, labels, average='macro'))

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(100,100,100), verbose=True)
model.fit(X_train_embed, y_train); labels = model.predict(X_test_embed)

In [ ]:
mat = confusion_matrix(y_test, labels)
plt.figure(figsize=(2,2))

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))

plt.xlabel('True Labeling'); plt.ylabel('Predicted Labeling')

In [ ]:
print('Microaveraged F1 score:', f1_score(y_test, labels, average='micro'))
print('Macroaveraged F1 score:', f1_score(y_test, labels, average='macro'))

In [ ]:
embeddings_index = {}
f = open('C:/Users/benja/Downloads/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
max_words = 10000; maxlen=100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

In [ ]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
max_features = 10000

model = Sequential()
model.add(Embedding(max_features, 100))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(X_train_embed, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_test_embed, y_test))

# Error: Cast string to float is not supported